<h1><center>LangChain Expression Language - LCEL</center></h1>
<hr><hr>
LCEL makes it easy to build complex chains from basic components, and supports out of the box functionality such as streaming, parallelism, and logging.

#### Runnable:
-----------------
A unit of work that can be invoked, batched, streamed, transformed and composed.<br>
Ex- A `Prompt`, `LLMChain` or any other type of chain, `ChatModel`(like `ChatOpenAI`, `AzureChatOpenAI`) are all examples of runnable, and they can be used/operated by the following methods.

- `invoke`/`ainvoke`: Transforms a single input into an output.
- `batch`/`abatch`: Efficiently transforms multiple inputs into outputs.
- `stream`/`astream`: Streams output from a single input as it’s produced.
- `astream_log`: Streams output and selected intermediate results from an input.

### Configurations:-
---------------------

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_api_key = os.getenv("AZURE_OPENAI_KEY")
azure_openai_api_version = "2023-05-15"
llm_deployment_name = os.getenv("GPT_DEPLOYMENT_NAME")

os.environ["OPENAI_API_TYPE"]     = "azure"
os.environ["OPENAI_API_VERSION"]  = azure_openai_api_version
os.environ["OPENAI_API_KEY"]      = azure_openai_api_key

## A. Basic example: prompt + model + output parser
-----------------------------------------------------
The most basic and common use case is chaining a prompt template and a model together. To see how this works, let’s create a chain that takes a topic and generates a joke:

In [91]:
from langchain_openai import AzureChatOpenAI

chat_model = AzureChatOpenAI(
    openai_api_version=azure_openai_api_version,
    azure_deployment=llm_deployment_name,
    temperature=0.7
)

In [92]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

joke_prompt = ChatPromptTemplate.from_template("""Tell me a joke about {joke_topic}""")
op_parser = StrOutputParser()

joke_chain = joke_prompt | chat_model | op_parser

### i. Using `.invoke()` method:
-----------------------------------

In [93]:
joke_topic = "Software Development"

response = joke_chain.invoke( {"joke_topic": joke_topic} )
print( response )

Why do programmers prefer dark mode?

Because light attracts bugs!


### ii. Using `.batch()` method:
-----------------------------------

In [94]:
response = joke_chain.batch( [{"joke_topic": "Cat"}, {"joke_topic": "England"}, {"joke_topic": "Cricket"}] )
print( response )

["Why don't cats play poker in the wild?\n\nToo many cheetahs!", 'Why did the English football team bring string to the game?\n\nBecause they wanted to tie the score!', 'Why did the cricket go to school?\n\nBecause it wanted to learn how to catch a fly ball!']


In [95]:
for _ in response:
    print(_, "\n\n\n")

Why don't cats play poker in the wild?

Too many cheetahs! 



Why did the English football team bring string to the game?

Because they wanted to tie the score! 



Why did the cricket go to school?

Because it wanted to learn how to catch a fly ball! 





### iii. Using `.stream()` method:
------------------------------------

In [99]:
stream_response = joke_chain.stream( {"joke_topic": "Women"} )
print( stream_response )

<generator object RunnableSequence.stream at 0x000001C92D606730>


In [101]:
for chunk in stream_response:
    print(chunk, end="_", flush=True)

Why_ did_ the_ girl_ bring_ a_ ladder_ to_ the_ bar_?

_Because_ she_ heard_ the_ drinks_ were_ on_ the_ house_!__

------------------------------------
The `|` symbol is similar to a **unix pipe operator**, which chains together the different components **feeds the output from one component as input into the next component**.

In this chain the user input is passed to the prompt template, then the prompt template output is passed to the model, then the model output is passed to the output parser. Let’s take a look at each component individually to really understand what’s going on.

### 1. Prompt:
----------------
- `prompt` is a `BasePromptTemplate`, which means it takes in a dictionary of template variables and produces a `PromptValue`.
- A `PromptValue` is a wrapper around a completed prompt that can be passed to either an LLM (which takes a string as input) or `ChatModel` (which takes a sequence of messages as input).
- It can work with either language model type because it defines logic both for producing `BaseMessages` and for producing a string.

In [12]:
prompt_value = joke_prompt.invoke({"joke_topic": "ice cream"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='Tell me a joke about ice cream')])

In [13]:
print( prompt_value )

messages=[HumanMessage(content='Tell me a joke about ice cream')]


In [14]:
prompt_value.to_messages()

[HumanMessage(content='Tell me a joke about ice cream')]

In [15]:
prompt_value.to_string()

'Human: Tell me a joke about ice cream'

### 2. Model:
---------------
- The `PromptValue` is then passed to model. In this case our model is a `ChatModel`, meaning it will output a `BaseMessage` (subcategory being `AIMessage`).
- If our model was an `LLM`, it would output a string.

In [16]:
message = chat_model.invoke( prompt_value )
message

AIMessage(content='Why did the ice cream go to therapy?\n\nBecause it had too many rocky road trips!')

In [17]:
print( message )

content='Why did the ice cream go to therapy?\n\nBecause it had too many rocky road trips!'


### 3. Output parser:
----------------------
And lastly we pass our model output to the `op_parser`, which is a `BaseOutputParser` meaning it takes either a `string` or a `BaseMessage` as input. The `StrOutputParser` specifically simple converts any input into a `string`.

In [19]:
op_parser.invoke( message )

'Why did the ice cream go to therapy?\n\nBecause it had too many rocky road trips!'

--------------

## B. RAG Search Example:
--------------------------
For our next example, we want to run a retrieval-augmented generation chain to add some context when responding to questions.

In [68]:
pip show faiss-cpu

Name: faiss-cpu
Version: 1.8.0
Summary: A library for efficient similarity search and clustering of dense vectors.
Home-page: 
Author: 
Author-email: Kota Yamaguchi <yamaguchi_kota@cyberagent.co.jp>
License: MIT License
Location: e:\programs & codes\generative ai\_genai_venv\lib\site-packages
Requires: numpy
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [25]:
pip show docarray

Name: docarray
Version: 0.40.0
Summary: The data structure for multimodal data
Home-page: https://docs.docarray.org/
Author: DocArray
Author-email: 
License: Apache 2.0
Location: e:\programs & codes\generative ai\_genai_venv\lib\site-packages
Requires: numpy, orjson, pydantic, rich, types-requests, typing-inspect
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [49]:
pip show langchain-text-splitters

Name: langchain-text-splitters
Version: 0.0.1
Summary: LangChain text splitting utilities
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: e:\programs & codes\generative ai\_genai_venv\lib\site-packages
Requires: langchain-core
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [87]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings

embedding_deployment_name = os.getenv("AZURE_OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME")

In [41]:
chat_model = AzureChatOpenAI(
    openai_api_version=azure_openai_api_version,
    azure_deployment=llm_deployment_name,
    temperature=0.1
)

embeddings = AzureOpenAIEmbeddings(
    azure_deployment=embedding_deployment_name,
    openai_api_version=azure_openai_api_version
)

In [82]:
raw_text_document = TextLoader('./data/001-context_data.txt').load()

raw_text_document

[Document(page_content='harrison worked at kensho\nbears like to eat honey\n', metadata={'source': './data/001-context_data.txt'})]

In [83]:
print( type(raw_text_document[0].page_content) )

<class 'str'>


In [89]:
text_splitter = CharacterTextSplitter(separator="\n")
# text_splitter = RecursiveCharacterTextSplitter()

texts = text_splitter.split_documents(raw_text_document)
texts

[Document(page_content='harrison worked at kensho\nbears like to eat honey', metadata={'source': './data/001-context_data.txt'})]

In [73]:
vectorstore = FAISS.from_documents( texts, embeddings )

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

chat_prompt = ChatPromptTemplate.from_template(template)

output_parser = StrOutputParser()

In [74]:
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

chat_chain = setup_and_retrieval | chat_prompt | chat_model | output_parser

In [75]:
response = chat_chain.invoke("where did harrison work?")
response

'Harrison worked at Kensho.'

To explain this, we first can see that the prompt template above takes in context and question as values to be substituted in the prompt. Before building the prompt template, we want to retrieve relevant documents to the search and include them as part of the context.

As a preliminary step, we’ve setup the `retriever` using an in memory store, which can retrieve documents based on a query. This is a runnable component as well that can be chained together with other components, but you can also try to run it separately:

In [77]:
response = retriever.invoke("where did harrison work?")
response

[Document(page_content='harrison worked at kensho\nbears like to eat honey', metadata={'source': './data/001-context_data.txt'})]

We then use the `RunnableParallel` to prepare the expected inputs into the prompt by using the entries for the retrieved documents as well as the original user question, using the retriever for document search, and `RunnablePassthrough` to pass the user’s question: